# **Extract, Transform and Load** (ETL)


In [1]:
import pandas as pd
import numpy as np

## Load

In [2]:
amazon_df = pd.read_csv("amazon_prime_titles.csv")
disney_df = pd.read_csv("disney_plus_titles.csv")
hulu_df = pd.read_csv("hulu_titles.csv")
netflix_df = pd.read_csv("netflix_titles.csv")

## Exploration

In [3]:
print('Amazon:', amazon_df.shape)
print('Disney:', disney_df.shape)
print('Hulu:', hulu_df.shape)
print('Netflix:', netflix_df.shape)

Amazon: (9668, 12)
Disney: (1450, 12)
Hulu: (3073, 12)
Netflix: (8807, 12)


In [4]:
#amazon_df.info()
#disney_df.info()
hulu_df.info() 
#netflix_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3073 entries, 0 to 3072
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   show_id       3073 non-null   object 
 1   type          3073 non-null   object 
 2   title         3073 non-null   object 
 3   director      3 non-null      object 
 4   cast          0 non-null      float64
 5   country       1620 non-null   object 
 6   date_added    3045 non-null   object 
 7   release_year  3073 non-null   int64  
 8   rating        2553 non-null   object 
 9   duration      2594 non-null   object 
 10  listed_in     3073 non-null   object 
 11  description   3069 non-null   object 
dtypes: float64(1), int64(1), object(10)
memory usage: 288.2+ KB


## Transformation

+ Generamos un campo **`id`** compuesto de la primera letra del nombre de la plataforma, seguido del show_id ya presente en los datasets (ejemplo para títulos de Amazon = **`as123`**)

In [5]:
''' Función que genera la columna 'id '''
def generar_id(df, letra):
    df['Aux'] = letra # Agregamos una nueva columna auxiliar con la primera letra de la plataforma correcpondiente
    df['id'] = df.Aux.str.cat(df.show_id) # Concatenamos las columnas 'Aux' con 'show_id" en una nueva columna 'id'
    df.drop(columns=['Aux'], inplace = True) # Eliminamos las columnas que ya no sirven    

In [6]:
''' Llamamos a la función 'generar_id' para cada data frame '''
generar_id(amazon_df,'a')  
generar_id(disney_df,'d')
generar_id(hulu_df,'h')
generar_id(netflix_df,'n')

+ Agregamos una columna **`platform`** para identificar la plataforma correspondiente 

In [7]:
amazon_df['platform'] = 'Amazon'
disney_df['platform'] = 'Disney'
hulu_df['platform'] = 'Hulu'
netflix_df['platform'] = 'Netflix'

* Unificamos los 4 DataFrame en uno solo al que llamaremos **`streaming_df`**

In [8]:
streaming_df = pd.concat([amazon_df, disney_df, hulu_df, netflix_df])

In [9]:
streaming_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22998 entries, 0 to 8806
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       22998 non-null  object
 1   type          22998 non-null  object
 2   title         22998 non-null  object
 3   director      14739 non-null  object
 4   cast          17677 non-null  object
 5   country       11499 non-null  object
 6   date_added    13444 non-null  object
 7   release_year  22998 non-null  int64 
 8   rating        22134 non-null  object
 9   duration      22516 non-null  object
 10  listed_in     22998 non-null  object
 11  description   22994 non-null  object
 12  id            22998 non-null  object
 13  platform      22998 non-null  object
dtypes: int64(1), object(13)
memory usage: 2.6+ MB


In [10]:
''' Cantidad de elementos núlos por columna '''
streaming_df.isnull().sum()

show_id             0
type                0
title               0
director         8259
cast             5321
country         11499
date_added       9554
release_year        0
rating            864
duration          482
listed_in           0
description         4
id                  0
platform            0
dtype: int64

* Cargamos el data set `score_set.csv` en el data frame **`score_df`**, el cual contiene el puntaje promedio para cada película y Tv Show identificados por `movieId`.

In [11]:
score_df = pd.read_csv('score_set.csv')
score_df.head()

,movieId,score(average)
0,as1,3.467131
1,as10,3.439571
2,as100,3.609302
3,as1000,3.556701
4,as1001,3.585288


In [12]:
score_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22998 entries, 0 to 22997
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   movieId         22998 non-null  object 
 1   score(average)  22998 non-null  float64
dtypes: float64(1), object(1)
memory usage: 359.5+ KB


* Hacemos el cruce de los DataFrame **`streaming_df`**  y **`score_df`** sobre las columnas `id` y `movieId` 

In [13]:
streaming_df = streaming_df.merge(score_df, left_on='id', right_on='movieId') 
streaming_df.head(2)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,id,platform,movieId,score(average)
0,s1,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",Canada,"March 30, 2021",2014,NaN,113 min,"Comedy, Drama",A small fishing village must procure a local d...,as1,Amazon,as1,3.467131
1,s2,Movie,Take Care Good Night,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",India,"March 30, 2021",2018,13+,110 min,"Drama, International",A Metro Family decides to fight a Cyber Crimin...,as2,Amazon,as2,3.548682


+ Los campos de texto deberán estar en **minúsculas**, sin excepciones

In [14]:
''' Function that transforms text fields to lower case '''
''' Función que transforma en minusculas los campos de texto '''

def conversion_minus(df):
    l = len(df.columns) - 1
    for i in range(0,l):
        if df[df.columns[i]].dtype == 'O':
            df[df.columns[i]] = df[df.columns[i]].str.lower()
        else:
            continue

In [15]:
conversion_minus(streaming_df)

In [16]:
streaming_df.head(2)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,id,platform,movieId,score(average)
0,s1,movie,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,"march 30, 2021",2014,NaN,113 min,"comedy, drama",a small fishing village must procure a local d...,as1,amazon,as1,3.467131
1,s2,movie,take care good night,girish joshi,"mahesh manjrekar, abhay mahajan, sachin khedekar",india,"march 30, 2021",2018,13+,110 min,"drama, international",a metro family decides to fight a cyber crimin...,as2,amazon,as2,3.548682


+ Las fechas, tendrán el formato **`AAAA-mm-dd`**

In [17]:
streaming_df['date_added'] = pd.to_datetime(streaming_df['date_added'], utc=False)

In [18]:
streaming_df['date_added'].head(4)

0   2021-03-30
1   2021-03-30
2   2021-03-30
3   2021-03-30
Name: date_added, dtype: datetime64[ns]


+ Los valores nulos del campo rating se reemplazaran por el string “**`G`**” (corresponde al maturity rating: “general for all audiences”)

In [19]:
streaming_df.rating.fillna('G', inplace = True)

In [20]:
streaming_df.isna().sum()

show_id               0
type                  0
title                 0
director           8259
cast               5321
country           11499
date_added         9554
release_year          0
rating                0
duration            482
listed_in             0
description           4
id                    0
platform              0
movieId               0
score(average)        0
dtype: int64

* Completemos los valores faltantes por una etiqueta `no_data` 

In [21]:
streaming_df.director.fillna('no_data', inplace=True)
streaming_df.cast.fillna('no_data', inplace=True)
streaming_df.country.fillna('no_data', inplace=True)
streaming_df.date_added.fillna('no_data', inplace=True)
streaming_df.description.fillna('no_data', inplace=True)

In [22]:
streaming_df.isna().sum()

show_id             0
type                0
title               0
director            0
cast                0
country             0
date_added          0
release_year        0
rating              0
duration          482
listed_in           0
description         0
id                  0
platform            0
movieId             0
score(average)      0
dtype: int64

+ El campo ***duration*** se dividirá en dos campos: **`duration_int`** y **`duration_type`**. El primero será un integer y el segundo un string indicando la unidad de medición de duración: min (minutos) o season (temporadas)

In [23]:
''' Primero reemplazamos estos valores NaN por la etiqueta `0 no_data` '''
streaming_df['duration'].fillna('0 no_data', inplace = True)

In [24]:
''' Dividimos la columna `duration` '''
streaming_df[['duration_int', 'duration_type']] = streaming_df['duration'].str.split(' ', expand=True)    

In [25]:
streaming_df.columns

Index(['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added',
       'release_year', 'rating', 'duration', 'listed_in', 'description', 'id',
       'platform', 'movieId', 'score(average)', 'duration_int',
       'duration_type'],
      dtype='object')

In [26]:
streaming_df['duration_type'].unique()

array(['min', 'season', 'seasons', 'no_data'], dtype=object)

In [27]:
''' Normalizaremos la columna duration_tipe '''
streaming_df['duration_type'].replace({"seasons": 'season'},inplace=True)
streaming_df['duration_type'].unique()

array(['min', 'season', 'no_data'], dtype=object)

In [28]:
''' Se observa que los valores etiquetados como `no_dato` corresponden solo a al tipo movie, por lo que estos serán reemplazados por la etiqueta `min`
 (duración que corresponde a películas)'''
print(streaming_df[streaming_df['duration_type'] == 'no_data']['type'].unique())


['movie']


In [29]:
streaming_df['duration_type'].replace({"no_data": 'min'},inplace=True)
streaming_df['duration_type'].unique()

array(['min', 'season'], dtype=object)

In [30]:
''' Convertimos a entero la columa 'duration_int'  '''
print(streaming_df['duration_int'].dtype)
streaming_df.duration_int = streaming_df.duration_int.astype("Int64")
print(streaming_df['duration_int'].dtype)

object
Int64


In [31]:
''' Verificamos que ya no haya más valores faltantes '''
streaming_df.isnull().sum()

show_id           0
type              0
title             0
director          0
cast              0
country           0
date_added        0
release_year      0
rating            0
duration          0
listed_in         0
description       0
id                0
platform          0
movieId           0
score(average)    0
duration_int      0
duration_type     0
dtype: int64

In [32]:
''' Eliminamos algunas columnas redundantes '''
streaming_df.drop(columns=['show_id'], inplace = True)
streaming_df.drop(columns=['duration'], inplace = True)
streaming_df.drop(columns=['movieId'], inplace = True)

In [33]:
streaming_df.columns

Index(['type', 'title', 'director', 'cast', 'country', 'date_added',
       'release_year', 'rating', 'listed_in', 'description', 'id', 'platform',
       'score(average)', 'duration_int', 'duration_type'],
      dtype='object')

In [34]:
''' Reordenamos las columnas '''
streaming_df = streaming_df[['id','title', 'director', 'cast','description','duration_int','duration_type',
                             'country','release_year','platform','type','listed_in','rating','score(average)','date_added']]
streaming_df.head(2)

,id,title,director,cast,description,duration_int,duration_type,country,release_year,platform,type,listed_in,rating,score(average),date_added
0,as1,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",a small fishing village must procure a local d...,113,min,canada,2014,amazon,movie,"comedy, drama",G,3.467131,2021-03-30 00:00:00
1,as2,take care good night,girish joshi,"mahesh manjrekar, abhay mahajan, sachin khedekar",a metro family decides to fight a cyber crimin...,110,min,india,2018,amazon,movie,"drama, international",13+,3.548682,2021-03-30 00:00:00


### Agregaremos dos columnas más: `country_main` y `score*1000`

La columna `country_main` contendrá el primer país que aparece en la columna `country` y será considerado como el país principal.

La columna `score*100` será en caso de no querer trabajar con decimales

In [35]:
streaming_df['country_main'] = streaming_df['country'].apply(lambda x: x.split(',')[0])

In [42]:
streaming_df['score*1000'] = streaming_df['score(average)'].apply(lambda x: round(x*10000,0))

In [44]:
streaming_df.columns

Index(['id', 'title', 'director', 'cast', 'description', 'duration_int',
       'duration_type', 'country', 'release_year', 'platform', 'type',
       'listed_in', 'rating', 'score(average)', 'date_added', 'country_main',
       'score*1000'],
      dtype='object')

* We export the DataFrame
* Exportamos el DataFrame

In [45]:
''' Finalmente exportamos '''
streaming_df.to_csv("streaming_set.csv", index=False) 